# PyTorch/CSPRNG encrypt/decrypt examples

torchcsprng 0.2.0 exposes new API for tensor encryption/decryption. Tensor encryption/decryption API is dtype agnostic, so a tensor of any dtype can be encrypted and the result can be stored to a tensor of any dtype. An encryption key also can be a tensor of any dtype. Currently torchcsprng supports AES cipher with 128-bit key in two modes: ECB and CTR.

In [ ]:
# !pip install --pre torchcsprng -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html
# for CUDA runtime:
!pip install --pre torchcsprng -f https://download.pytorch.org/whl/nightly/cu101/torch_nightly.html

In [ ]:
import torch
import torchcsprng as csprng

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torchcsprng implementation of AES with 128 bit key requires to have a key tensor of 16 bytes but of any dtype

In [ ]:
key = torch.empty(16, dtype=torch.uint8, device=device).random_(0, 256)
key

Alternatively it can be a tensor of 8 elements of `torch.int16` or even 4 elements of `torch.float32`

The size of input tensor is 42 * (32/8) = 168 bytes. AES 128 operates with 16-bytes blocks, so zero-padding of 8 bytes will be used to form 176 bytes(eleven 16-bytes blocks)

In [ ]:
initial = torch.empty(42, dtype=torch.float32, device=device).normal_(-24.0, 42.0)
initial

torchcsprng requires output tensor to be of the same size in bytes as input tensor rounded up to 16 bytes(AES 128 block size), so if `torch.int64` is dtype of the destination tensor size must be 176 / (64/8) = 22

In [ ]:
encrypted = torch.empty(22, dtype=torch.int64, device=device)

Call `torchcsprng.encrypt` to encrypt `initial` tensor in [ECB](https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation#Electronic_codebook_(ECB)) mode with 128-bit `key` tensor and store the result to `encrypted` tensor.

In [ ]:
csprng.encrypt(initial, encrypted, key, "aes128", "ecb")

Create an output tensor

In [ ]:
decrypted = torch.empty_like(initial)

Call `torchcsprng.decrypt` to decrypt `encrypted` tensor in ECB mode with 128-bit `key` tensor and store the result to `decrypted` tensor.

In [ ]:
csprng.decrypt(encrypted, decrypted, key, "aes128", "ecb")